In [1]:
import pandas as pd
import MySQLdb
from datetime import datetime
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [7]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [ ]:
db.close()

In [23]:
dbname = 'bu_oi_data'
cursor = db.cursor()

name = '`中天期货`'

sql = "ALTER TABLE " + dbname + ".v_rank ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_rank ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_rank ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_increment ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_increment ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_increment ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_volume ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_oi ADD COLUMN " + name + "TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_oi ADD COLUMN " + name + "TEXT"
cursor.execute(sql)

1580

In [20]:
dbname = 'bu_oi_data'
cursor = db.cursor()
text = "'2018-05-11'"

sql = "delete from " + dbname + ".total_volume where Date >= " + text 
cursor.execute(sql)
sql = "delete from " + dbname + ".futures_member where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".none_futures_member where Date >= " + text

sql = "delete from " + dbname + ".v_rank where Date >= " + text 
cursor.execute(sql)
sql = "delete from " + dbname + ".b_rank where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".s_rank where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".v_volume where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".b_oi where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".s_oi where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".v_increment where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".b_increment where Date >= " + text
cursor.execute(sql)
sql = "delete from " + dbname + ".s_increment where Date >= " + text
cursor.execute(sql)

1

In [10]:
# contract_id=['ag','al','au','bu','fu','hc','ni','pb','sn','zn','ru','cu','rb']
contract_id=['ag']
FOLDER_DIR = "../各交易所期貨持倉量/上期所/"

for x in contract_id:
    
    print(x)
    
    company_order = pd.read_excel('../期貨公司名稱/上期所/' + x.lower() + '_company_name.xlsx',encoding='gbk',header=None,index_col=None)
    cursor = db.cursor()
    DATA_DIR = FOLDER_DIR + x
    
    dbname1 = x + "_oi_original_data"
    dbname2 = x + "_oi_data"
    
#     V_info_create(dbname2, company_order)
#     B_info_create(dbname2, company_order)
#     S_info_create(dbname2, company_order)
    
#     sql = "CREATE DATABASE IF NOT EXISTS " + dbname1
#     cursor.execute(sql)
#     sql = "CREATE DATABASE IF NOT EXISTS " + dbname2
#     cursor.execute(sql)
    
#     create_futures_member(dbname2)
#     create_non_futures_member(dbname2)
#     create_total_volume(dbname2)
    
    for file in os.listdir(DATA_DIR):

        split_name = file.split("_")
        getname = split_name[0]
        date = split_name[1].split(".csv")[0]
        
        date = datetime.strptime(date,'%Y%m%d')

        if (date == datetime(2018,3,5)):
            
            df = pd.read_csv(DATA_DIR+"/"+ file,engine="python",encoding="utf8",header=1,index_col=0)
            df = df.where((pd.notnull(df)), None)
            date = date.strftime('%Y-%m-%d')
            
            if df.loc[1][1] == '期货公司':

#                 total_insert(df, file.split(".csv")[0], dbname1, 3)

                futures_member_insert(df, date, getname, dbname2,1)
                non_futures_member_insert(df, date, getname, dbname2,1)
#                 total_volume_insert(df, date, getname, dbname2)

#                 V_info_insert(df, getname, date, dbname2)
#                 B_info_insert(df, getname, date, dbname2)
#                 S_info_insert(df, getname, date, dbname2)

            else:
                if(getname.find("all") == 2):
#                     print(1)
                    futures_member_insert(df, date, getname, dbname2,0)
                    non_futures_member_insert(df, date, getname, dbname2,0)
                else:      
                    V_info_insert(df, getname, date, dbname2)
                    B_info_insert(df, getname, date, dbname2)
                    S_info_insert(df, getname, date, dbname2)
                    total_volume_insert(df, date, getname, dbname2)
#                     total_insert(df, file.split(".csv")[0], dbname1, 1)

ag


IntegrityError: (1062, "Duplicate entry '2018-03-05-ag1803' for key 'PRIMARY'")

In [19]:
b = company_order.values.tolist()

a = df['期货公司会员简称.1']
a = pd.DataFrame(list(a))
a = a.values.tolist()

for i in a:
    if i not in b:
        print(i)

['中天期货']
['-871']
[None]


In [3]:
def total_insert(data,tbname,databaseName,start_index):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + "." + tbname + "(\
    Rank INT(5),\
    Member CHAR(10),\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Rank_1 INT(5),\
    Member1 CHAR(10),\
    Buy_Volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Rank_2 INT(5),\
    Member2 CHAR(10),\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5)) ENGINE = MyISAM"

    cursor.execute(sql)
    value=[]

    for i in range(start_index,len(data)-1):      
        value.append((data.loc[i][0],data.loc[i][1],data.loc[i][2],data.loc[i][3],data.loc[i][4],data.loc[i][5],data.loc[i][6],data.loc[i][7],data.loc[i][8],data.loc[i][9],data.loc[i][10],data.loc[i][11]))

    sql = "INSERT INTO " + databaseName + "." + tbname +" VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

In [4]:
def create_futures_member(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".futures_member (\
    Date DATE ,\
    Contract_id TEXT(5),\
    Total_volume INT(5),\
    Increment INT(5),\
    Total_buy_volume INT(5),\
    Increment_1 INT(5),\
    Total_sales_volume INT(5),\
    Increment_2 INT(5) ) "

    cursor.execute(sql)

def futures_member_insert(data, table_date, tbname, databaseName,index):
    cursor = db.cursor()
    
    sql = "INSERT INTO " + databaseName + ".futures_member VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
    if index == 1:
        cursor.execute(sql,(table_date,tbname,data.loc[1][2],data.loc[1][3],data.loc[1][6],data.loc[1][7],\
                       data.loc[1][10],data.loc[1][11])) 
    else:
        cursor.execute(sql,(table_date,tbname,data.loc[1][1],data.loc[1][2],data.loc[1][4],data.loc[1][5],\
                       data.loc[1][7],data.loc[1][8])) 
    db.commit()
    cursor.close()
    
def create_non_futures_member(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".none_futures_member (\
    Date DATE ,\
    Contract_id TEXT(5),\
    Total_volume INT(5),\
    Increment INT(5),\
    Total_buy_volume INT(5),\
    Increment_1 INT(5),\
    Total_sales_volume INT(5),\
    Increment_2 INT(5) ) "

    cursor.execute(sql)
    
def non_futures_member_insert(data, table_date, tbname, databaseName,index):
    cursor = db.cursor()
    sql = "INSERT INTO " + databaseName + ".none_futures_member VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
    
    if index == 1:
        cursor.execute(sql,(table_date,tbname,data.loc[2][2],data.loc[2][3],data.loc[2][6],data.loc[1][7],\
                   data.loc[2][10],data.loc[2][11])) 
    else:
        cursor.execute(sql,(table_date,tbname,data.loc[2][1],data.loc[2][2],data.loc[2][4],data.loc[2][5],\
                   data.loc[2][7],data.loc[2][8])) 

    db.commit()
    cursor.close()
    
def create_total_volume(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume ( Date DATE , Contract_id TEXT(5), \
    V_Valume INT(5),V_Increment INT(5),\
    B_Valume INT(5),B_Increment INT(5),\
    S_Valume INT(5),S_Increment INT(5))"

    cursor.execute(sql)
    
def total_volume_insert(data, table_date, tbname, databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
     
    if(len(data) > 2):   
        sql = "INSERT INTO " + databaseName + ".total_volume VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.execute(sql,(table_date,tbname,data.loc[total_len][1],data.loc[total_len][2],\
                            data.loc[total_len][4],data.loc[total_len][5],\
                            data.loc[total_len][7],data.loc[total_len][8]))
        db.commit()
        cursor.close()

In [5]:
def V_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE, Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def V_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"


    sql = "INSERT INTO " + databaseName + ".v_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None or data.loc[i][1] == '期货公司' or data.loc[i][1] == '非期货公司'):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]) + "=" + str(data.loc[i][0]) + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None or data.loc[i][1] == '期货公司' or data.loc[i][1] == '非期货公司'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]) + "=" + str(data.loc[i][2]) + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None or data.loc[i][1] == '期货公司' or data.loc[i][1] == '非期货公司'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + str(data.loc[i][1]) + "=" + str(data.loc[i][3]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

def B_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def B_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
 
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][5] == None or data.loc[i][5] == '期货公司' or data.loc[i][5] == '非期货公司'):
            continue  
        else:
            sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][5]) + "=" + str(data.loc[i][4]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][5] == None or data.loc[i][5] == '期货公司' or data.loc[i][5] == '非期货公司'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][5]) + "=" + str(data.loc[i][6]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][5] == None or data.loc[i][5] == '期货公司' or data.loc[i][5] == '非期货公司'):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".b_increment SET " + str(data.loc[i][5]) + "=" + str(data.loc[i][7]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()

## S名次、賣單量、增減

def S_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def S_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()

    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][9] == None or data.loc[i][9] == '期货公司' or data.loc[i][9] == '非期货公司'):
            continue  
        
        else:
            sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][9]) + "=" + str(data.loc[i][8]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][9] == None or data.loc[i][9] == '期货公司' or data.loc[i][9] == '非期货公司'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][9]) + "=" + str(data.loc[i][10]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][9] == None or data.loc[i][9] == '期货公司' or data.loc[i][9] == '非期货公司'):
            continue
            
        else:
            sql = "UPDATE " + databaseName +".s_increment SET " + str(data.loc[i][9]) + "=" + str(data.loc[i][11]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()

# 找交易所名稱

In [ ]:
contract_id=['AG','AL','AU','BU','CU','FU','HC','NI','PB','RB','RU','SN','ZN']

In [ ]:
db = MySQLdb.connect("localhost","root","123456","sn_oi_original_data")
db.set_character_set('gbk')

In [ ]:
cursor = db.cursor()
sql = 'show tables'
cursor.execute(sql)
data = cursor.fetchall()

In [ ]:
company_list = []
for table in data:
    sql = 'select Member,Member1,Member2 from ' + table[0]
    cursor.execute(sql)
    name = cursor.fetchall()   
    for i in name:
        for j in range(3):
            if (i[j] not in company_list and i[j]!=None):
                company_list.append(i[j])

In [ ]:
a = pd.DataFrame(company_list)
a.to_excel("../期貨公司名稱/sn_company_name.xlsx",index=None,header=None)